In [1]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

from labellines import labelLine, labelLines

from centrex_tlf import states, hamiltonian, transitions, couplings, lindblad, utils
from centrex_tlf_julia_extension import lindblad_julia


c:\Users\ogras\anaconda3\envs\centrex-tlf\lib\site-packages\julia\juliainfo.py:93: UserWarning: julia warned:
The latest version of Julia in the `release` channel is 1.10.5+0.x64.w64.mingw32. You currently have `1.10.4+0.x64.w64.mingw32` installed. Run:

  juliaup update

in your terminal shell to install Julia 1.10.5+0.x64.w64.mingw32 and update the `release` channel to that version.
  warnings.warn("{} warned:\n{}".format(julia, stderr))


In [2]:
plt.rcParams.update({"font.size": 15})

In [3]:
trans = [
    transitions.OpticalTransition(
        transitions.OpticalTransitionType.R, J_ground=0, F1=1 / 2, F=1
    ),
]

polarizations = [
    [couplings.polarization_Z, couplings.polarization_X],
]

transition_selectors = couplings.generate_transition_selectors(
    trans,
    polarizations,
)


In [4]:
v_to_Γ = utils.detuning.velocity_to_detuning(1)

odepars = lindblad_julia.odeParameters(
    Ω0="Ωl0",
    Ωl0=1 * hamiltonian.Γ,  # Rabi frequency of the laser [rad/s]
    δ0=f"vx*{v_to_Γ}",  # detuning of the laser [rad/s]
    # laser phase modulation
    ω0=2 * np.pi * 1.56e6,  # polarization switching frequency [rad/s]
    φ0=0.0,  # polarization switching phase [rad]
    PX0="square_wave(t, ω0, ϕ0)",
    PZ0="1-PX0",
    # molecules
    vz=184,  # longitudinal molecular velocity [m/s]
    vx=0,
)


In [5]:
obe_system = lindblad.setup_OBE_system_transitions(
    trans,
    transition_selectors,
    verbose=True,
    qn_compact=True,
    decay_channels=None,
    Jmax_X=3,
)


INFO:centrex_tlf.lindblad.utils_setup:generate_OBE_system: 1/5 -> Generating the reduced Hamiltonian
INFO:centrex_tlf.lindblad.utils_setup:generate_OBE_system: 2/5 -> Generating the couplings corresponding to the transitions
INFO:centrex_tlf.lindblad.utils_setup:generate_OBE_system: 3/5 -> Generating the symbolic Hamiltonian
INFO:centrex_tlf.lindblad.utils_setup:generate_OBE_system: 4/5 -> Generating the collapse matrices
INFO:centrex_tlf.lindblad.utils_setup:generate_OBE_system: 5/5 -> Transforming the Hamiltonian and collapse matrices into a symbolic system of equations


In [6]:
obe_system.H_symbolic

Matrix([
[                        δ0,                          0,                          0,                          0,                0, -0.353553390506226*PX0*Ω0,                0.5*PZ0*Ω0, 0.353553390680278*PX0*Ω0],
[                         0,      δ0 - 83566.1617128386,                          0,                          0,                0,  -0.52192360261906*PZ0*Ω0, -0.369055718632921*PX0*Ω0,                        0],
[                         0,                          0,      δ0 - 83566.3655475656,                          0,                0, -0.369055718684215*PX0*Ω0,                         0, -0.36905571860351*PX0*Ω0],
[                         0,                          0,                          0,      δ0 - 83566.5693822376,                0,                         0, -0.369055718632921*PX0*Ω0, 0.521923602504928*PZ0*Ω0],
[                         0,                          0,                          0,                          0, 256817005329.751,             

In [7]:
[qn.largest for qn in obe_system.QN]

[|X, J = 0, F₁ = 1/2, F = 0, mF = 0, I₁ = 1/2, I₂ = 1/2, P = +, Ω = 0>,
 |X, J = 0, F₁ = 1/2, F = 1, mF = -1, I₁ = 1/2, I₂ = 1/2, P = +, Ω = 0>,
 |X, J = 0, F₁ = 1/2, F = 1, mF = 0, I₁ = 1/2, I₂ = 1/2, P = +, Ω = 0>,
 |X, J = 0, F₁ = 1/2, F = 1, mF = 1, I₁ = 1/2, I₂ = 1/2, P = +, Ω = 0>,
 |X, J = 2, F₁ = None, F = None, mF = None, I₁ = 1/2, I₂ = 1/2, P = +, Ω = 0>,
 |B, J = 1, F₁ = 1/2, F = 1, mF = -1, I₁ = 1/2, I₂ = 1/2, P = -, Ω = 1>,
 |B, J = 1, F₁ = 1/2, F = 1, mF = 0, I₁ = 1/2, I₂ = 1/2, P = -, Ω = 1>,
 |B, J = 1, F₁ = 1/2, F = 1, mF = 1, I₁ = 1/2, I₂ = 1/2, P = -, Ω = 1>]

In [10]:
obe_system = lindblad_julia.setup_OBE_system_julia(
    obe_system, transition_selectors, odepars
)


In [11]:
couplings.generate_br_dataframe(obe_system.ground, obe_system.excited, group_ground="J")

,"|B, J = 1, F₁ = 1/2, F = 1>"
states,
"|X, J = 0>",0.666454
"|X, J = 2>",0.333546


In [29]:
couplings.generate_br_dataframe(
    obe_system.ground, obe_system.excited, group_excited=False
).head()


,"|B, J = 1, F₁ = 1/2, F = 1, mF = -1>","|B, J = 1, F₁ = 1/2, F = 1, mF = 0>","|B, J = 1, F₁ = 1/2, F = 1, mF = 1>"
states,,,
"|X, J = 0, F₁ = 1/2, F = 0, mF = 0>",0.069876,0.069876,0.069876
"|X, J = 0, F₁ = 1/2, F = 1, mF = -1>",0.076138,0.076138,0.000000
"|X, J = 0, F₁ = 1/2, F = 1, mF = 0>",0.076138,0.000000,0.076138
"|X, J = 0, F₁ = 1/2, F = 1, mF = 1>",0.000000,0.076138,0.076138
"|X, J = 2, F₁ = 3/2, F = 1, mF = -1>",0.008326,0.008326,0.000000


In [27]:
couplings.generate_coupling_dataframe(obe_system.couplings[0], obe_system.QN)[0]

,ground,excited,couplings
0,"|X, J = 0, F₁ = 1/2, F = 0, mF = 0>","|B, J = 1, F₁ = 1/2, F = 1, mF = 0>",-0.457851+0.000000j
1,"|X, J = 0, F₁ = 1/2, F = 1, mF = -1>","|B, J = 1, F₁ = 1/2, F = 1, mF = -1>",0.477926+0.000000j
2,"|X, J = 0, F₁ = 1/2, F = 1, mF = 1>","|B, J = 1, F₁ = 1/2, F = 1, mF = 1>",-0.477926+0.000000j


In [28]:
couplings.generate_coupling_dataframe(obe_system.couplings[0], obe_system.QN)[1]

,ground,excited,couplings
0,"|X, J = 0, F₁ = 1/2, F = 0, mF = 0>","|B, J = 1, F₁ = 1/2, F = 1, mF = -1>",0.323749+0.000000j
1,"|X, J = 0, F₁ = 1/2, F = 0, mF = 0>","|B, J = 1, F₁ = 1/2, F = 1, mF = 1>",-0.323749+0.000000j
2,"|X, J = 0, F₁ = 1/2, F = 1, mF = -1>","|B, J = 1, F₁ = 1/2, F = 1, mF = 0>",0.337945+0.000000j
3,"|X, J = 0, F₁ = 1/2, F = 1, mF = 0>","|B, J = 1, F₁ = 1/2, F = 1, mF = -1>",0.337945+0.000000j
4,"|X, J = 0, F₁ = 1/2, F = 1, mF = 0>","|B, J = 1, F₁ = 1/2, F = 1, mF = 1>",0.337945+0.000000j
5,"|X, J = 0, F₁ = 1/2, F = 1, mF = 1>","|B, J = 1, F₁ = 1/2, F = 1, mF = 0>",0.337945+0.000000j
